# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Topic Modeling - Attempt #1 (All Text)

In [1]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

aaaaah  aaah  aah  abc  abcs  ability  abject  able  ablebodied  \
ali           0     0    0    1     0        0       0     2           0   
anthony       0     0    0    0     0        0       0     0           0   
bill          1     0    0    0     1        0       0     1           0   
dave          0     1    0    0     0        0       0     0           0   
jim           0     0    0    0     0        0       0     1           2   
joe           0     0    0    0     0        0       0     2           0   
john          0     0    0    0     0        0       0     3           0   
louis         0     0    3    0     0        0       0     1           0   
mike          0     0    0    0     0        0       0     0           0   
ricky         0     0    0    0     0        1       1     2           0   

         abortion  ...  ze  zealand  zeppelin  zero  zillion  zombie  zombies  \
ali             0  ...   0        0         0     0        0       1        0   
anthony         2  ...   0       10         0     0        0       0        0   
bill            0  ...   1        0         0     1        1       1        1   
dave            0  ...   0        0         0     0        0       0        0   
jim             0  ...   0        0         0     0        0       0        0   
joe             0  ...   0        0         0     0        0       0        0   
john            0  ...   0        0         0     0        0       0        0   
louis           0  ...   0        0         0     2        0       0        0   
mike            0  ...   0        0         2     1        0       0        0   
ricky           0  ...   0        0         0     0        0       0        0   

         zoning  zoo  éclair  
ali           0    0       0  
anthony       0    0       0  
bill          1    0       0  
dave          0    0       0  
jim           0    0       0  
joe           0    0       0  
john          0    0       1  
louis         0    0       0  
mike          0    0       0  
ricky         0    1       0  

[10 rows x 6749 columns]

In [2]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

ali  anthony  bill  dave  jim  joe  john  louis  mike  ricky
aaaaah    0        0     1     0    0    0     0      0     0      0
aaah      0        0     0     1    0    0     0      0     0      0
aah       0        0     0     0    0    0     0      3     0      0
abc       1        0     0     0    0    0     0      0     0      0
abcs      0        0     1     0    0    0     0      0     0      0

coverted the sparse matrix in gensim corpus format

In [4]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [31]:
sparse_counts

<6749x10 sparse matrix of type '<class 'numpy.int64'>'
	with 13889 stored elements in Compressed Sparse Row format>

In [32]:
corpus

location of all the words in cv_stop.pkl in document term matrix

In [5]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [33]:
id2word

{3299: 'ladies',
 2469: 'gentlemen',
 6531: 'welcome',
 5610: 'stage',
 135: 'ali',
 6635: 'wong',
 2777: 'hi',
 2753: 'hello',
 5969: 'thank',
 1205: 'coming',
 2528: 'gonna',
 5281: 'shit',
 923: 'cause',
 4310: 'pee',
 3424: 'like',
 3759: 'minutes',
 2017: 'everybody',
 1206: 'comingum',
 2038: 'exciting',
 1511: 'day',
 6705: 'year',
 6219: 'turned',
 6714: 'yes',
 4324: 'people',
 248: 'appreciate',
 6264: 'uh',
 5929: 'tell',
 2951: 'im',
 2481: 'getting',
 4102: 'older',
 2493: 'girl',
 367: 'automatic',
 6009: 'thought',
 2390: 'fuck',
 1752: 'dont',
 3277: 'know',
 5703: 'straight',
 3128: 'jealous',
 2328: 'foremost',
 3713: 'metabolism',
 2496: 'girls',
 3205: 'just',
 1873: 'eat',
 5381: 'sixpack',
 4949: 'right',
 2541: 'got',
 5978: 'thatthat',
 497: 'beautiful',
 3019: 'inner',
 5998: 'thigh',
 1109: 'clearance',
 2174: 'feet',
 5991: 'theres',
 2896: 'huge',
 2430: 'gap',
 3420: 'light',
 4520: 'potential',
 4726: 'radiating',
 6027: 'throughand',
 5412: 'sleep',
 3030

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

num_topics (int, optional) – The number of requested latent topics to be extracted from the training corpus.
passes (int, optional) – Number of passes through the corpus during training.

In [39]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.035*"like" + 0.016*"know" + 0.016*"im" + 0.016*"just" + 0.013*"dont" + 0.009*"thats" + 0.008*"said" + 0.007*"youre" + 0.006*"gonna" + 0.006*"think"'),
 (1,
  '0.023*"like" + 0.015*"just" + 0.015*"im" + 0.013*"dont" + 0.013*"right" + 0.013*"know" + 0.009*"people" + 0.009*"thats" + 0.008*"got" + 0.008*"fucking"')]

In [7]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.031*"like" + 0.019*"just" + 0.015*"know" + 0.013*"dont" + 0.013*"im" + 0.010*"people" + 0.010*"right" + 0.010*"thats" + 0.008*"gonna" + 0.008*"youre"'),
 (1,
  '0.001*"like" + 0.001*"im" + 0.001*"just" + 0.001*"dont" + 0.001*"right" + 0.001*"know" + 0.001*"people" + 0.000*"youre" + 0.000*"thats" + 0.000*"gonna"'),
 (2,
  '0.026*"like" + 0.019*"im" + 0.014*"know" + 0.014*"dont" + 0.013*"just" + 0.011*"right" + 0.009*"said" + 0.009*"thats" + 0.007*"youre" + 0.006*"got"')]

In [8]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.031*"like" + 0.016*"just" + 0.014*"know" + 0.014*"im" + 0.013*"dont" + 0.013*"right" + 0.009*"got" + 0.008*"thats" + 0.008*"gonna" + 0.007*"youre"'),
 (1,
  '0.023*"like" + 0.017*"just" + 0.013*"know" + 0.013*"im" + 0.012*"dont" + 0.012*"people" + 0.010*"right" + 0.010*"thats" + 0.008*"youre" + 0.007*"said"'),
 (2,
  '0.017*"like" + 0.015*"im" + 0.014*"dont" + 0.012*"right" + 0.012*"fucking" + 0.010*"just" + 0.010*"know" + 0.010*"went" + 0.007*"youre" + 0.007*"people"'),
 (3,
  '0.037*"like" + 0.022*"im" + 0.017*"know" + 0.015*"just" + 0.014*"dont" + 0.009*"said" + 0.009*"thats" + 0.006*"think" + 0.006*"youre" + 0.006*"gonna"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

if the word starts with 'NN' then it is noun.
so we check nouns in the text.
alos nouns can be checked using NNS,NNPS,NNP.

In [66]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)
    

In [67]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

transcript
ali       ladies and gentlemen please welcome to the st...
anthony   thank you thank you thank you san francisco t...
bill       all right thank you thank you very much than...
dave      this is dave he tells dirty jokes for a livin...
jim          ladies and gentlemen please welcome to the...
joe       ladies and gentlemen welcome joe roganwhat th...
john      armed with boyish charm and a sharp wit the f...
louis     introfade the music out lets roll hold there ...
mike      wow hey thank you thanks thank you guys hey s...
ricky     hello hello how you doing great thank you wow...

In [68]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean['transcript'].apply(nouns))
data_nouns
#data_clean['transcript'].apply(nouns)
#nouns(data_clean.transcript)

transcript
ali      ladies gentlemen stage ali hi thank hello na s...
anthony  thank thank people i em i francisco city world...
bill     thank thank pleasure georgia area oasis i june...
dave     jokes living stare work profound train thought...
jim      ladies gentlemen stage mr jim jefferies thank ...
joe      ladies gentlemen joe fck thanks phone fckface ...
john     charm wit snl writer john mulaney marriage bee...
louis    music lets lights lights thank i i place place...
mike     wow hey thanks look insane years everyone i id...
ricky    hello thank fuck thank im gon youre weve money...

In [69]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words="english")
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

aah  abc  abcs  ability  abortion  abortions  abuse  abusedi  accent  \
ali        0    1     0        0         0          0      0        1       0   
anthony    0    0     0        0         2          0      0        0       1   
bill       0    0     1        0         0          0      0        0       0   
dave       0    0     0        0         0          1      0        0       0   
jim        0    0     0        0         0          0      0        0       4   
joe        0    0     0        0         0          0      1        0       1   
john       0    0     0        0         0          0      0        0       1   
louis      3    0     0        0         0          0      0        0       0   
mike       0    0     0        0         0          0      0        0       0   
ricky      0    0     0        1         0          0      0        0       0   

         accents  ...  yulin  yummy  ze  zealand  zeppelin  zillion  zombie  \
ali            0  ...      0      0   0        0         0        0       1   
anthony        0  ...      0      0   0       10         0        0       0   
bill           0  ...      0      0   1        0         0        1       1   
dave           0  ...      0      0   0        0         0        0       0   
jim            0  ...      0      0   0        0         0        0       0   
joe            1  ...      0      0   0        0         0        0       0   
john           0  ...      0      0   0        0         0        0       0   
louis          0  ...      0      0   0        0         0        0       0   
mike           0  ...      0      0   0        0         2        0       0   
ricky          0  ...      1      1   0        0         0        0       0   

         zombies  zoo  éclair  
ali            0    0       0  
anthony        0    0       0  
bill           1    0       0  
dave           0    0       0  
jim            0    0       0  
joe            0    0       0  
john           0    0       1  
louis          0    0       0  
mike           0    0       0  
ricky          0    1       0  

[10 rows x 4147 columns]

In [70]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [71]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.019*"people" + 0.014*"im" + 0.013*"youre" + 0.012*"thats" + 0.011*"day" + 0.009*"thing" + 0.007*"time" + 0.007*"dont" + 0.006*"man" + 0.006*"shit"'),
 (1,
  '0.016*"im" + 0.014*"people" + 0.012*"thats" + 0.012*"youre" + 0.011*"time" + 0.008*"shit" + 0.008*"thing" + 0.007*"cause" + 0.007*"guy" + 0.007*"dont"')]

In [72]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.023*"people" + 0.015*"im" + 0.013*"youre" + 0.013*"shit" + 0.012*"thats" + 0.011*"time" + 0.010*"man" + 0.008*"thing" + 0.008*"day" + 0.007*"guy"'),
 (1,
  '0.013*"youre" + 0.013*"people" + 0.012*"thats" + 0.012*"im" + 0.010*"time" + 0.009*"thing" + 0.009*"day" + 0.008*"hes" + 0.008*"life" + 0.007*"dont"'),
 (2,
  '0.020*"im" + 0.010*"thats" + 0.009*"cause" + 0.009*"youre" + 0.008*"people" + 0.008*"dont" + 0.008*"day" + 0.006*"thing" + 0.006*"point" + 0.006*"guy"')]

In [73]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.017*"people" + 0.013*"time" + 0.013*"shit" + 0.013*"thats" + 0.013*"youre" + 0.012*"im" + 0.010*"lot" + 0.009*"life" + 0.008*"hes" + 0.008*"man"'),
 (1,
  '0.019*"im" + 0.013*"people" + 0.012*"day" + 0.011*"thats" + 0.011*"youre" + 0.009*"thing" + 0.009*"time" + 0.009*"cause" + 0.008*"dont" + 0.007*"way"'),
 (2,
  '0.022*"people" + 0.015*"youre" + 0.014*"im" + 0.013*"thats" + 0.009*"shit" + 0.009*"thing" + 0.008*"man" + 0.008*"right" + 0.008*"hes" + 0.008*"dont"'),
 (3,
  '0.000*"youre" + 0.000*"thats" + 0.000*"im" + 0.000*"people" + 0.000*"thing" + 0.000*"cause" + 0.000*"right" + 0.000*"day" + 0.000*"dont" + 0.000*"hes"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [74]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [75]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

transcript
ali      ladies gentlemen welcome stage ali wong hi wel...
anthony  thank san francisco thank good people surprise...
bill     right thank thank pleasure greater atlanta geo...
dave     dirty jokes living stare most hard work profou...
jim      ladies gentlemen welcome stage mr jim jefferie...
joe      ladies gentlemen joe fck san francisco thanks ...
john     boyish charm sharp wit former snl writer john ...
louis    music lets lights lights thank much i i i nice...
mike     wow hey thanks hey seattle nice look crazy ins...
ricky    hello great thank fuck thank lovely welcome im...

In [76]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words="english", max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

aaaaah  aah  abc  abcs  ability  abject  able  ablebodied  abortion  \
ali           0    0    1     0        0       0     2           0         0   
anthony       0    0    0     0        0       0     0           0         2   
bill          1    0    0     1        0       0     1           0         0   
dave          0    0    0     0        0       0     0           0         0   
jim           0    0    0     0        0       0     1           2         0   
joe           0    0    0     0        0       0     2           0         0   
john          0    0    0     0        0       0     3           0         0   
louis         0    3    0     0        0       0     1           0         0   
mike          0    0    0     0        0       0     0           0         0   
ricky         0    0    0     0        1       1     2           0         0   

         abortions  ...  yummy  yyou  ze  zealand  zeppelin  zillion  zombie  \
ali              0  ...      0     1   0        0         0        0       1   
anthony          0  ...      0     0   0       10         0        0       0   
bill             0  ...      0     0   1        0         0        1       1   
dave             1  ...      0     0   0        0         0        0       0   
jim              0  ...      0     0   0        0         0        0       0   
joe              0  ...      0     0   0        0         0        0       0   
john             0  ...      0     0   0        0         0        0       0   
louis            0  ...      0     0   0        0         0        0       0   
mike             0  ...      0     0   0        0         2        0       0   
ricky            0  ...      1     0   0        0         0        0       0   

         zombies  zoo  éclair  
ali            0    0       0  
anthony        0    0       0  
bill           1    0       0  
dave           0    0       0  
jim            0    0       0  
joe            0    0       0  
john           0    0       1  
louis          0    0       0  
mike           0    0       0  
ricky          0    1       0  

[10 rows x 5041 columns]

In [77]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [78]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.007*"dude" + 0.005*"fck" + 0.005*"kid" + 0.005*"fucking" + 0.004*"fuck" + 0.004*"everybody" + 0.004*"joke" + 0.003*"different" + 0.003*"water" + 0.003*"fcking"'),
 (1,
  '0.005*"fuck" + 0.004*"fucking" + 0.004*"kid" + 0.003*"friend" + 0.003*"joke" + 0.003*"ahah" + 0.003*"mom" + 0.003*"parents" + 0.003*"men" + 0.003*"sure"')]

In [79]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.007*"fck" + 0.007*"dude" + 0.006*"kid" + 0.005*"everybody" + 0.004*"mom" + 0.004*"fuck" + 0.004*"fucking" + 0.004*"wife" + 0.004*"dick" + 0.004*"fcking"'),
 (1,
  '0.004*"ok" + 0.004*"joke" + 0.004*"jenny" + 0.004*"course" + 0.003*"different" + 0.003*"uh" + 0.003*"kid" + 0.003*"nuts" + 0.003*"dead" + 0.003*"tit"'),
 (2,
  '0.009*"fuck" + 0.007*"fucking" + 0.006*"ahah" + 0.006*"joke" + 0.005*"guns" + 0.005*"men" + 0.004*"son" + 0.004*"anthony" + 0.004*"family" + 0.004*"mad"')]

In [80]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.009*"joke" + 0.007*"anthony" + 0.006*"course" + 0.006*"tit" + 0.005*"kid" + 0.005*"grandma" + 0.004*"shark" + 0.004*"ok" + 0.004*"family" + 0.004*"jokes"'),
 (1,
  '0.008*"fuck" + 0.008*"fucking" + 0.005*"dude" + 0.005*"gun" + 0.004*"kid" + 0.004*"jenny" + 0.004*"guns" + 0.004*"uh" + 0.004*"class" + 0.004*"ass"'),
 (2,
  '0.006*"fck" + 0.005*"dude" + 0.005*"fuck" + 0.005*"ahah" + 0.004*"different" + 0.004*"water" + 0.004*"fucking" + 0.004*"men" + 0.004*"everybody" + 0.004*"joke"'),
 (3,
  '0.009*"clinton" + 0.008*"mom" + 0.007*"cow" + 0.006*"wife" + 0.005*"parents" + 0.005*"kid" + 0.004*"president" + 0.004*"movie" + 0.004*"john" + 0.004*"friend"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [81]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.014*"joke" + 0.011*"anthony" + 0.007*"grandma" + 0.006*"jokes" + 0.006*"family" + 0.006*"shark" + 0.006*"mad" + 0.005*"san" + 0.005*"mom" + 0.004*"kid"'),
 (1,
  '0.005*"fuck" + 0.005*"ahah" + 0.005*"mom" + 0.005*"ok" + 0.004*"kid" + 0.004*"wife" + 0.004*"clinton" + 0.003*"different" + 0.003*"gay" + 0.003*"dude"'),
 (2,
  '0.009*"joke" + 0.005*"nuts" + 0.005*"jenner" + 0.005*"hampstead" + 0.005*"chimp" + 0.004*"rape" + 0.004*"funny" + 0.004*"fucking" + 0.004*"twitter" + 0.004*"tweet"'),
 (3,
  '0.006*"fuck" + 0.006*"fucking" + 0.006*"dude" + 0.006*"fck" + 0.005*"kid" + 0.005*"girl" + 0.004*"everybody" + 0.004*"gun" + 0.004*"ass" + 0.003*"men"')]

These four topics look pretty decent. Let's settle on these for now.
* Topic 0: mom, parents
* Topic 1: husband, wife
* Topic 2: guns
* Topic 3: profanity

In [82]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusn]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(1, 'ali'),
 (0, 'anthony'),
 (3, 'bill'),
 (1, 'dave'),
 (3, 'jim'),
 (3, 'joe'),
 (1, 'john'),
 (1, 'louis'),
 (3, 'mike'),
 (2, 'ricky')]

For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
* Topic 0: mom, parents [Ali,joe,louis]
* Topic 1: husband, wife [anthony,bill,dave,john]
* Topic 2: guns [guns]
* Topic 3: profanity [jim,mike]

### Assignment:
1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.

QUESTION 1:

In [86]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordna, passes=100)
ldana.print_topics()

[(0,
  '0.009*"fck" + 0.006*"dude" + 0.006*"joke" + 0.005*"kid" + 0.005*"mom" + 0.004*"everybody" + 0.004*"fcking" + 0.004*"anthony" + 0.004*"husband" + 0.004*"girl"'),
 (1,
  '0.006*"fuck" + 0.006*"fucking" + 0.005*"joke" + 0.005*"dude" + 0.005*"kid" + 0.004*"course" + 0.004*"nuts" + 0.004*"everybody" + 0.003*"tit" + 0.003*"hell"'),
 (2,
  '0.006*"fuck" + 0.005*"fucking" + 0.004*"ahah" + 0.004*"friend" + 0.003*"wife" + 0.003*"uh" + 0.003*"men" + 0.003*"jenny" + 0.003*"son" + 0.003*"sure"')]

For the first set of words:

Topic 0: Family and Relationships
Topic 1: Expletives and Vulgar Language
Topic 2: Humor and Jokes
For the second set of words:

Topic 0: Expletives and Vulgar Language
Topic 1: Humor and Jokes
Topic 2: Personal Relationships and Interactions

QUESTION 2:

In [49]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def adverbs(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_adverb = lambda pos: pos[:2] == 'RB'
    tokenized = word_tokenize(text)
    all_adverbs = [word for (word, pos) in pos_tag(tokenized) if is_adverb(pos)] 
    return ' '.join(all_adverbs)

In [50]:
data_adverbs = pd.DataFrame(data_clean['transcript'].apply(adverbs))
data_adverbs

transcript
ali      so very very yes because now dont even just th...
anthony  so much so here most politically not im not do...
bill     very much here here here here ridiculously the...
dave     very much here very im not ever detroit well v...
jim      very sweet right now very now so very really n...
joe      im na not not right not also not not yet just ...
john     sly there pretty again very now then so too to...
louis    very necessarily very much well easily around ...
mike     here so ago pretty really sometimes so distinc...
ricky    just not just well actually not obviously they...

In [56]:
# Create a new document-term matrix using only adverbs also remove common words with max_df
cvna = CountVectorizer(stop_words="english", max_df=.8)
data_cvna = cvna.fit_transform(data_adverbs.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_adverbs.index
data_dtmna

abroad  absolutely  accidentally  admittedly  aggressively  ago  ah  \
ali           0           0             1           0             1    1   0   
anthony       0           0             1           0             0    1   0   
bill          0           3             2           0             0    2   0   
dave          0           0             0           0             0    2   0   
jim           3           0             0           0             0    0   0   
joe           0           0             1           0             0    0   0   
john          0           1             0           0             0    2   2   
louis         0           0             0           0             0    2   0   
mike          0           0             0           0             0    1   0   
ricky         0           0             0           1             0    5   1   

         ahead  alarmingly  ali  ...  ya  yeahcourtyard  yep  yes  yoga  yolo  \
ali          1           1    1  ...   0              0    0    4     1     1   
anthony      0           0    0  ...   0              0    0    0     0     0   
bill         1           0    0  ...   1              0    0    0     0     0   
dave         1           0    0  ...   0              0    0    2     0     0   
jim          0           0    0  ...   0              0    0    1     0     0   
joe          1           0    0  ...   1              0    0    0     0     0   
john         0           0    0  ...   0              0    0    3     0     0   
louis        1           0    0  ...   0              1    0    3     0     0   
mike         0           0    0  ...   0              0    0    1     0     0   
ricky        0           0    0  ...   0              0    1    6     0     0   

         youd  youll  youre  youve  
ali         0      0      0      2  
anthony     1      0      0      1  
bill        1      0      1      0  
dave        1      1      0      0  
jim         0      0      2      1  
joe         1      0      3      2  
john        1      0      1      0  
louis       1      0      4      0  
mike        2      0      3      0  
ricky       0      0      4      1  

[10 rows x 330 columns]

In [57]:
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [58]:
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.035*"anymore" + 0.027*"basically" + 0.022*"finally" + 0.022*"ta" + 0.020*"pretty" + 0.020*"im" + 0.018*"immediately" + 0.018*"especially" + 0.016*"yes" + 0.014*"literally"'),
 (1,
  '0.043*"ta" + 0.033*"definitely" + 0.026*"exactly" + 0.022*"totally" + 0.015*"constantly" + 0.015*"youre" + 0.015*"oh" + 0.015*"im" + 0.015*"anymore" + 0.012*"slowly"'),
 (2,
  '0.029*"im" + 0.023*"obviously" + 0.021*"oh" + 0.020*"soon" + 0.020*"yes" + 0.018*"youre" + 0.016*"suddenly" + 0.016*"ago" + 0.016*"pretty" + 0.014*"early"')]

Topic 0: This topic appears to contain adverbs that express some level of uncertainty, such as "anymore," "eventually," and "totally." It also includes adverbs that express some level of completeness or finality, such as "completely" and "finally."

Topic 1: This topic seems to include adverbs that express some level of certainty or agreement, such as "definitely," "exactly," and "yes." It also includes adverbs that express some level of timelessness or permanence, such as "forever."

Topic 2: This topic appears to contain adverbs that express some level of immediacy or suddenness, such as "soon," "suddenly," and "immediately." It also includes adverbs that express some level of obviousness, such as "obviously."